<a href="https://colab.research.google.com/github/amandatz/computational-linear-algebra/blob/main/Atividade2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 2

In [15]:
using LinearAlgebra

using Printf

function print_matrix(M)
    for i in 1:size(M, 1)
        for j in 1:size(M, 2)
            @printf("%10.4f ", M[i, j])
        end
        println()
    end
end


print_matrix (generic function with 1 method)

## Questão 1

In [8]:
function back_substitution(U, b)
    n = length(b)
    x = zeros(n)

    for i in n:-1:1
        sum_term = sum(U[i, j] * x[j] for j in i+1:n; init=0.0)
        x[i] = (b[i] - sum_term) / U[i, i]
    end

    return x
end

back_substitution (generic function with 1 method)

In [9]:
function forward_substitution(L, b)
    n = length(b)
    x = zeros(n)

    for i in 1:n
        sum_term = sum(L[i, j] * x[j] for j in 1:i-1; init=0.0)
        x[i] = (b[i] - sum_term) / L[i, i]
    end

    return x
end


forward_substitution (generic function with 1 method)

In [19]:
n = 4
U = triu(rand(n, n))
L = tril(rand(n, n))
b = rand(n)

x_back = back_substitution(U, b)
x_forward = forward_substitution(L, b)

println("Upper Triangular Matrix U:")
print_matrix(U)
println("Solução (back-substitution):")
print_matrix(x_back)

println("\nLower Triangular Matrix L:")
print_matrix(L)
println("Solução (forward-substitution):")
print_matrix(x_forward)


Upper Triangular Matrix U:
    0.5951     0.6238     0.1677     0.7195 
    0.0000     0.8411     0.6168     0.3838 
    0.0000     0.0000     0.4039     0.7787 
    0.0000     0.0000     0.0000     0.5427 
Solução (back-substitution):
   -1.6888 
    1.0156 
   -0.8490 
    0.9189 

Lower Triangular Matrix L:
    0.1366     0.0000     0.0000     0.0000 
    0.0872     0.0762     0.0000     0.0000 
    0.7903     0.8308     0.1244     0.0000 
    0.4685     0.4529     0.7464     0.6596 
Solução (forward-substitution):
    1.0783 
    7.7392 
  -55.5392 
   57.5278 


## Questão 2

## Questão 3

## Questão 4